# Mise en place du notebook

## Versionning

P6 v7 : avant split entre "P6 part1 v01" (contient la récupération des données et le SIFT) et "P6 part2 v01" (contient VGG-16).  
P6 part2 : début VGG-16.
P6 part2 v03 : version ayant servi à la création de part 3.  
P6 part3 v01 : tf-idf sur les data.  
P6 part3 v02 : cleaning des data.    

## Import des modules

In [181]:
# imports pour le développement
from pympler import asizeof
import mesfonctions
from importlib import reload
mesfonctions = reload(mesfonctions)

# imports pour la production
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import adjusted_rand_score
import pickle
from termcolor import colored, cprint


# Pour NLP
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer


## Fonctions courantes

In [3]:
def nom_fichier_photo(index, df=None):
    """Retourne le chemin + nom du fichier jpg d'une ligne de dataframe.
    Args : 
    - index : index de la ligne de df.
    Returns :
    - string.
    """
    if df is None:
        df = df_source
    return PATH + df['image'][index]

## Constantes

In [4]:
PATH = "Images_Flipkart/"

## Gestion des paramètres d'affichage de Jupyter

In [5]:
from IPython.core.display import HTML
plt.style.use('seaborn')

# pour gérer l'affichage des dataframes pandas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200
pd.set_option("colheader_justify", "left")

# pour gérer l'affichage des arrays numpy
# 230 sur config bureau, 165 sur config portable
np.set_printoptions(linewidth=165)

# pour afficher les gros fichiers sur toute la largeur de l'écran
HTML("<style>.container { width:95% }</style>")

# Chargement du dataframe

In [6]:
df_source = pickle.load(open('df_source.pkl', 'rb'))

In [7]:
df_source

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications,label
0,55b85ea15a1536d46b7190ad6fff8ce7,2016-04-30 03:22:56 +0000,http://www.flipkart.com/elegance-polyester-mul...,Elegance Polyester Multicolor Abstract Eyelet ...,"[""Home Furnishing >> Curtains & Accessories >>...",CRNEG7BKMFFYHQ8Z,1899.0,899.0,55b85ea15a1536d46b7190ad6fff8ce7.jpg,False,Key Features of Elegance Polyester Multicolor ...,No rating available,No rating available,Elegance,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
1,7b72c92c2f6c40268628ec5f14c6d590,2016-04-30 03:22:56 +0000,http://www.flipkart.com/sathiyas-cotton-bath-t...,Sathiyas Cotton Bath Towel,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEGFZHGBXPHZUH,600.0,449.0,7b72c92c2f6c40268628ec5f14c6d590.jpg,False,Specifications of Sathiyas Cotton Bath Towel (...,No rating available,No rating available,Sathiyas,"{""product_specification""=>[{""key""=>""Machine Wa...",Baby Care
2,64d5d4a258243731dc7bbb1eef49ad74,2016-04-30 03:22:56 +0000,http://www.flipkart.com/eurospa-cotton-terry-f...,Eurospa Cotton Terry Face Towel Set,"[""Baby Care >> Baby Bath & Skin >> Baby Bath T...",BTWEG6SHXTDB2A2Y,NaN,NaN,64d5d4a258243731dc7bbb1eef49ad74.jpg,False,Key Features of Eurospa Cotton Terry Face Towe...,No rating available,No rating available,Eurospa,"{""product_specification""=>[{""key""=>""Material"",...",Baby Care
3,d4684dcdc759dd9cdf41504698d737d8,2016-06-20 08:49:52 +0000,http://www.flipkart.com/santosh-royal-fashion-...,SANTOSH ROYAL FASHION Cotton Printed King size...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJT9UQWHDUBH4,2699.0,1299.0,d4684dcdc759dd9cdf41504698d737d8.jpg,False,Key Features of SANTOSH ROYAL FASHION Cotton P...,No rating available,No rating available,SANTOSH ROYAL FASHION,"{""product_specification""=>[{""key""=>""Brand"", ""v...",Home Furnishing
4,6325b6870c54cd47be6ebfbffa620ec7,2016-06-20 08:49:52 +0000,http://www.flipkart.com/jaipur-print-cotton-fl...,Jaipur Print Cotton Floral King sized Double B...,"[""Home Furnishing >> Bed Linen >> Bedsheets >>...",BDSEJTHNGWVGWWQU,2599.0,698.0,6325b6870c54cd47be6ebfbffa620ec7.jpg,False,Key Features of Jaipur Print Cotton Floral Kin...,No rating available,No rating available,Jaipur Print,"{""product_specification""=>[{""key""=>""Machine Wa...",Home Furnishing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,958f54f4c46b53c8a0a9b8167d9140bc,2015-12-01 10:15:43 +0000,http://www.flipkart.com/oren-empower-extra-lar...,Oren Empower Extra Large Self Adhesive Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE88ZGTX65GH4V,1399.0,999.0,958f54f4c46b53c8a0a9b8167d9140bc.jpg,False,Oren Empower Extra Large Self Adhesive Sticker...,No rating available,No rating available,Oren Empower,"{""product_specification""=>[{""key""=>""Number of ...",Baby Care
1046,fd6cbcc22efb6b761bd564c28928483c,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallmantra-large-vinyl...,Wallmantra Large Vinyl Sticker Sticker,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIEC889ZD5GDCVQ,4930.0,1896.0,fd6cbcc22efb6b761bd564c28928483c.jpg,False,Wallmantra Large Vinyl Sticker Sticker (Pack o...,No rating available,No rating available,Wallmantra,"{""product_specification""=>[{""key""=>""Number of ...",Baby Care
1047,5912e037d12774bb73a2048f35a00009,2015-12-01 10:15:43 +0000,http://www.flipkart.com/uberlyfe-extra-large-p...,Uberlyfe Extra Large Pigmented Polyvinyl Films...,"[""Baby Care >> Baby & Kids Gifts >> Stickers >...",STIE5UVGW2JWVWCT,4500.0,1449.0,5912e037d12774bb73a2048f35a00009.jpg,False,Buy Uberlyfe Extra Large Pigmented Polyvinyl F...,No rating available,No rating available,Uberlyfe,"{""product_specification""=>[{""key""=>""Number of ...",Baby Care
1048,c3edc504d1b4f0ba6224fa53a43a7ad6,2015-12-01 10:15:43 +0000,http://www.flipkart.com/wallman

# Feature engineering

J'avais dans l'idée de créer un texte basé sur les variables product_name et product_specifications, mais c'est inutile car ils ont déjà été concaténés dans la variable description (note : le 861 y est bien ; c'est dû à une erreur dans le product_name, rectifié dans description).

In [8]:
for i in range(len(df_source)):
    if not (df_source['product_name'][i] in df_source['description'][i]):
        print(f"Il manque le produit numéro {i} dans description.")

Il manque le produit numéro 861 dans description.


In [153]:
descriptions = df_source['description'].copy()

# Nettoyage du texte

Inspiré de : https://machinelearningmastery.com/clean-text-machine-learning-python/

In [208]:
def nettoyage_document(text, output='tokens'):
    """ Tokenization et nettoyage d'un document :
    1) tokenization,
    2) casse = minuscule,
    3) suppression de la ponctuation,
    4) suppression des termes non alphabétiques,
    5) suppression des stop words,
    6) stemming.
    Args :
    - text : string (document) à nettoyer.
    Returns :
    - texte nettoyé (sous forme de liste de strings (tokens), ou de string reconcaténé)
    """
    # split into words
    from nltk.tokenize import word_tokenize
    tokens = word_tokenize(text)
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    # remove punctuation from each word
    import string
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    # filter out stop words
    from nltk.corpus import stopwords
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if w not in stop_words]
    # stemming of words
    from nltk.stem.porter import PorterStemmer
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in words]
    
    # return
    if output == 'tokens':
        return stemmed
    else:
        return ' '.join(stemmed)


In [213]:
# test unitaire
texte = descriptions[42]
print(texte)
print('-'*50)
nettoyage_document(texte, output='string')

Key Features of Mom and Kid Baby Girl's Printed Blue, Grey Top & Pyjama Set Fabric: Cotton Brand Color: Blue, Grey,Mom and Kid Baby Girl's Printed Blue, Grey Top & Pyjama Set Price: Rs. 309 Girls Pyjamaset,Specifications of Mom and Kid Baby Girl's Printed Blue, Grey Top & Pyjama Set General Details Pattern Printed Ideal For Baby Girl's Night Suit Details Number of Contents in Sales Package Pack of 1 Fabric Cotton Type Top & Pyjama Set Neck Round Nack In the Box 1 Top & Pyjama Set
--------------------------------------------------


'key featur mom kid babi girl print blue grey top pyjama set fabric cotton brand color blue grey mom kid babi girl print blue grey top pyjama set price rs girl pyjamaset specif mom kid babi girl print blue grey top pyjama set gener detail pattern print ideal babi girl night suit detail number content sale packag pack fabric cotton type top pyjama set neck round nack box top pyjama set'

In [194]:
descriptions_stemmed = descriptions.copy().map(lambda texte : nettoyage_document(texte, output='string'))

In [212]:
descriptions_stemmed[42]

'key featur mom kid babi girl print blue grey top pyjama set fabric cotton brand color blue grey mom kid babi girl print blue grey top pyjama set price rs girl pyjamaset specif mom kid babi girl print blue grey top pyjama set gener detail pattern print ideal babi girl night suit detail number content sale packag pack fabric cotton type top pyjama set neck round nack box top pyjama set'

# Clustering du dataset description avec TF-IDF

Inspiré de :  https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html

## Dataset non nettoyé

### CountVectorizer()

In [114]:
vectorizer = CountVectorizer(
#                              max_features=500,
                             stop_words='english',
                             min_df=10,  # le terme doit être présent dans >=10 documents
                             max_df=0.1   # le terme doit être présent dans <=10% des documents
                             )
vectorized = vectorizer.fit_transform(descriptions)

print("Nb de features :", vectorizer.get_feature_names_out().shape[0], "\n")

print("Features :\n", vectorizer.get_feature_names_out(), "\n")

print(vectorized.toarray())

vectorized

Nb de features : 522 

Features :
 ['10' '100' '12' '13' '14' '15' '150' '16' '17' '175' '18' '19' '199' '20' '200' '220' '225' '250' '299' '300' '329' '330' '349' '350' '399' '40' '45' '499' '50'
 '549' '550' '599' '5v' '60' '699' '75' '750' '799' '800' '899' '90' '999' 'abstract' 'ac' 'accessories' 'actual' 'adapter' 'add' 'additional' 'adorable'
 'affordable' 'age' 'air' 'aluminium' 'antique' 'apple' 'apply' 'area' 'aroma' 'art' 'assured' 'attractive' 'audio' 'authentication' 'available' 'baby' 'bag' 'base'
 'based' 'bath' 'battery' 'beautiful' 'beautifully' 'beauty' 'bed' 'bedroom' 'bedsheet' 'black' 'blanket' 'bleach' 'blue' 'body' 'bottle' 'boy' 'boys' 'brass'
 'break' 'bring' 'brings' 'broadcast' 'brown' 'buckle' 'bulb' 'button' 'cable' 'capacity' 'care' 'carry' 'case' 'casual' 'catchy' 'caused' 'ce' 'cell' 'center'
 'ceramic' 'certifications' 'charge' 'charger' 'check' 'checkered' 'cheer' 'clasp' 'classic' 'clean' 'closure' 'cloth' 'cms' 'code' 'coffee' 'cold' 'collection'
 'co

<1050x522 sparse matrix of type '<class 'numpy.int64'>'
	with 12541 stored elements in Compressed Sparse Row format>

In [106]:
print(vectorized)

  (0, 370)	6
  (0, 325)	4
  (0, 42)	4
  (0, 192)	6
  (0, 172)	5
  (0, 146)	9
  (0, 203)	1
  (0, 241)	2
  (0, 39)	1
  (0, 294)	1
  (0, 1)	1
  (0, 243)	1
  (0, 460)	1
  (0, 313)	1
  (0, 304)	1
  (0, 407)	2
  (0, 183)	2
  (0, 246)	2
  (0, 160)	1
  (0, 466)	1
  (0, 135)	1
  (0, 320)	2
  (0, 223)	1
  (0, 516)	1
  (0, 449)	1
  :	:
  (1046, 504)	2
  (1046, 59)	2
  (1046, 254)	2
  (1046, 397)	2
  (1046, 182)	4
  (1046, 278)	1
  (1046, 496)	4
  (1046, 456)	4
  (1046, 493)	3
  (1046, 56)	8
  (1046, 492)	2
  (1046, 377)	2
  (1046, 188)	2
  (1046, 186)	2
  (1046, 436)	2
  (1046, 244)	2
  (1046, 418)	2
  (1047, 278)	2
  (1047, 456)	2
  (1048, 310)	2
  (1048, 456)	4
  (1048, 493)	2
  (1049, 278)	2
  (1049, 456)	2
  (1049, 493)	2


In [103]:
terms = vectorizer.get_feature_names_out()
terms[520]

'yellow'

In [100]:
len(vectorizer.stop_words_)

5321

### TfidfTransformer()

In [107]:
transformer = TfidfTransformer()
X = transformer.fit_transform(vectorized)
X

<1050x522 sparse matrix of type '<class 'numpy.float64'>'
	with 12541 stored elements in Compressed Sparse Row format>

In [109]:
transformer.idf_

array([4.31991121, 3.9144461 , 4.46098981, 5.5596021 , 4.82200315, 4.62529286, 5.06712561, 4.9617651 , 5.24944717, 5.47259072, 4.9617651 , 5.39254801, 4.26861792,
       5.31844004, 5.47259072, 5.5596021 , 5.24944717, 5.18490865, 4.40214931, 4.86645492, 5.47259072, 5.06712561, 5.39254801, 4.49176147, 4.46098981, 5.24944717,
       5.5596021 , 4.62529286, 4.6616605 , 5.39254801, 5.39254801, 4.6616605 , 5.5596021 , 5.12428403, 5.01305839, 5.24944717, 5.47259072, 5.01305839, 5.12428403,
       5.24944717, 5.39254801, 4.37397843, 4.19629726, 5.24944717, 5.06712561, 5.06712561, 5.24944717, 4.59020154, 3.51484611, 5.18490865, 5.24944717, 5.31844004,
       5.5596021 , 5.31844004, 5.47259072, 5.47259072, 5.06712561, 5.12428403, 5.39254801, 4.73862155, 5.31844004, 4.73862155, 5.47259072, 5.39254801, 4.9617651 ,
       3.36237752, 5.18490865, 5.39254801, 5.5596021 , 4.55629999, 4.17330774, 4.2439253 , 4.77944354, 5.47259072, 4.9617651 , 5.18490865, 5.39254801, 3.65343228,
       4.59020154, 4.4

### TfidfVectorizer()

In [132]:
# TfidfVectorizer fait en one-pot CountVectorizer et TfidfTransformer

# le meilleur ARI est obtenu en filtrant le moins de mots (pas de stop words)
vectorizer = TfidfVectorizer(
#                              max_features=500,
#                              stop_words='english',
#                              min_df=10,  # le terme doit être présent dans >=10 documents
#                              max_df=0.1   # le terme doit être présent dans <=10% des documents
                             )

X = vectorizer.fit_transform(descriptions)
print("n_samples: %d, n_features: %d" % X.shape)
X

n_samples: 1050, n_features: 6053


<1050x6053 sparse matrix of type '<class 'numpy.float64'>'
	with 46143 stored elements in Compressed Sparse Row format>

In [149]:
terms = vectorizer.get_feature_names_out()
terms[2001]

'defined'

### KMeans()

In [182]:
km = KMeans(n_clusters=7, init='k-means++', max_iter=1000, n_init=100)
km.fit(X)
cprint("Adjusted Rand Index: %.2f" %
      adjusted_rand_score(df_source['label'], km.labels_), 'red')

Adjusted Rand Index: 0.29


In [133]:
km.cluster_centers_

array([[0.00020061, 0.00034868, 0.00076091, ..., 0.00020939, 0.00096018, 0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        , 0.00647802]])

In [134]:
km.cluster_centers_.argsort()  # les coordonnées (des centres des clusters) sont triées de la plus petite à la plus grande

array([[6052, 1064, 3011, ..., 5497,  939, 3984],
       [   0, 3976, 3975, ..., 3678,  934, 5874],
       [   0, 4013, 4012, ..., 2576, 1702, 1061],
       ...,
       [   0, 3938, 3937, ..., 5007, 1231, 4408],
       [   0, 3952, 3951, ..., 1540, 5566, 3804],
       [   0, 3919, 3918, ..., 4019, 2576, 1702]], dtype=int64)

In [135]:
order_centroids = km.cluster_centers_.argsort()[:, ::-1]   # les coordonnées (des centres des clusters) sont triées de la plus grande à la plus petite
order_centroids

array([[3984,  939, 5497, ..., 3011, 1064, 6052],
       [5874,  934, 3678, ..., 3975, 3976,    0],
       [1061, 1702, 2576, ..., 4012, 4013,    0],
       ...,
       [4408, 1231, 5007, ..., 3937, 3938,    0],
       [3804, 5566, 1540, ..., 3951, 3952,    0],
       [1702, 2576, 4019, ..., 3918, 3919,    0]], dtype=int64)

In [152]:
for i in range(7):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print()

Cluster 0: of and the to cm in your is pack with
Cluster 1: watch analog men at only discounts women india great for
Cluster 2: at com flipkart genuine shipping cash delivery buy free products
Cluster 3: baby girl details of fabric dress boy cotton sleeve neck
Cluster 4: prices best showpiece with online cm at 30 guarantee day
Cluster 5: mug to ceramic coffee and your it perfect mugs rockmantra
Cluster 6: com flipkart only from guarantee 30 replacement day shipping genuine


Trop de mots inutiles ici : le fait de conserver tous les mots pour maximiser l'ARI conduit à avoir des "stop words" dans les mots clés.

## Dataset nettoyé